In [ ]:
# Code for Encryption

from Crypto.Cipher import AES, DES3
from Crypto.Util import Counter
import binascii

def even_position_left_shift(input_bytes):
    """Derived keys k1, k2, k3 via even-position left shift [cite: 1612-1615]."""
    bit_string = ''.join(format(b, '08b') for b in input_bytes)
    even_bits = bit_string[1::2] 
    odd_bits = bit_string[0::2]  
    combined = even_bits + odd_bits
    return int(combined, 2).to_bytes(len(input_bytes), byteorder='big')

def modular_addition(msg, nonce_prime):
    """Combines message and modified nonce [cite: 1628-1629]."""
    result = bytearray()
    for i in range(len(msg)):
        val = (msg[i] + nonce_prime[i % len(nonce_prime)]) % 256
        result.append(val)
    return bytes(result)

def encrypt_modified_eax(K, N, M, H):
    # Step 2: Key Derivation - Split original key K [cite: 1601-1604]
    half = len(K) // 2
    k_a, k_b = K[:half], K[half:]
    
    # Step 3: 3DES-ECB to generate L [cite: 1607-1609]
    des3_key = DES3.adjust_key_parity(k_a + k_b + k_a)
    cipher_3des = DES3.new(des3_key, DES3.MODE_ECB)
    L = cipher_3des.encrypt(N[:8]) 
    
    # Step 4: Chained key derivation for k1, k2, k3 [cite: 1612-1615]
    k1 = even_position_left_shift(L)
    k2 = even_position_left_shift(k1)
    k3 = even_position_left_shift(k2)
    
    # Step 5: Nonce Auth & N' generation [cite: 1617-1621]
    cipher_cbc0 = AES.new(k1.ljust(16, b'\0'), AES.MODE_CBC, iv=bytes(16))
    Tag1 = cipher_cbc0.encrypt(L.ljust(16, b'\0'))
    N_prime = Tag1 
    
    # Step 6: Header Auth (Tag2) [cite: 1622-1626]
    cipher_cbc1 = AES.new(k2.ljust(16, b'\0'), AES.MODE_CBC, iv=bytes(16))
    Tag2 = cipher_cbc1.encrypt(H.ljust(16, b'\0')[:16])
    
    # Step 7: CTR Encryption with modular addition [cite: 1627-1632]
    M_prime = modular_addition(M, N_prime)
    cipher_ctr = AES.new(K, AES.MODE_CTR, counter=Counter.new(128))
    C = cipher_ctr.encrypt(M_prime)
    
    # Step 8: Ciphertext Auth (Tag3) [cite: 1633-1636]
    cipher_cbc2 = AES.new(k3.ljust(16, b'\0'), AES.MODE_CBC, iv=bytes(16))
    Tag3 = cipher_cbc2.encrypt(C.ljust(16, b'\0')[:16])
    
    # Step 9: Final XOR Tag [cite: 1638-1642]
    T = bytes(a ^ b ^ c for a, b, c in zip(Tag1, Tag2, Tag3))
    return C, T

# Example Inputs [cite: 1662-1665]
key = binascii.unhexlify("0f1571c947d9e8590cb7add6af7f6798")
nonce = binascii.unhexlify("00112233445566778899aabbccddeeff")
msg = b"Hello World!"
hdr = b"MyProject"

ciphertext, tag = encrypt_modified_eax(key, nonce, msg, hdr)
print(f"ENCRYPTION RESULT:\nCiphertext: {binascii.hexlify(ciphertext).decode()}\nTag: {binascii.hexlify(tag).decode()}")

In [ ]:
# Code for Decryption

from Crypto.Cipher import AES, DES3
from Crypto.Util import Counter
import binascii

# Re-use helper functions from Encryption for Key Derivation and Modular logic
def even_position_left_shift(input_bytes):
    bit_string = ''.join(format(b, '08b') for b in input_bytes)
    combined = bit_string[1::2] + bit_string[0::2]
    return int(combined, 2).to_bytes(len(input_bytes), byteorder='big')

def modular_subtraction(ciphertext_processed, nonce_prime):
    """Reverses modular addition for decryption."""
    result = bytearray()
    for i in range(len(ciphertext_processed)):
        val = (ciphertext_processed[i] - nonce_prime[i % len(nonce_prime)]) % 256
        result.append(val)
    return bytes(result)

def decrypt_modified_eax(K, N, C, H, received_T):
    # Step 1: Re-derive all keys and Tags for verification [cite: 1554]
    half = len(K) // 2
    k_a, k_b = K[:half], K[half:]
    des3_key = DES3.adjust_key_parity(k_a + k_b + k_a)
    L = DES3.new(des3_key, DES3.MODE_ECB).encrypt(N[:8])
    
    k1 = even_position_left_shift(L)
    k2 = even_position_left_shift(k1)
    k3 = even_position_left_shift(k2)
    
    # Recalculate Tags to verify integrity [cite: 1588, 1736]
    Tag1 = AES.new(k1.ljust(16, b'\0'), AES.MODE_CBC, iv=bytes(16)).encrypt(L.ljust(16, b'\0'))
    Tag2 = AES.new(k2.ljust(16, b'\0'), AES.MODE_CBC, iv=bytes(16)).encrypt(H.ljust(16, b'\0')[:16])
    Tag3 = AES.new(k3.ljust(16, b'\0'), AES.MODE_CBC, iv=bytes(16)).encrypt(C.ljust(16, b'\0')[:16])
    
    # Verify Authentication Tag [cite: 1641]
    calculated_T = bytes(a ^ b ^ c for a, b, c in zip(Tag1, Tag2, Tag3))
    
    if calculated_T != received_T:
        raise ValueError("INTEGRITY ERROR: Data has been tampered with! [cite: 1588]")
    
    # Step 2: Decrypt Ciphertext [cite: 990, 1113]
    cipher_ctr = AES.new(K, AES.MODE_CTR, counter=Counter.new(128))
    M_prime = cipher_ctr.decrypt(C)
    
    # Step 3: Reverse modular addition [cite: 1558]
    original_M = modular_subtraction(M_prime, Tag1)
    return original_M

# Example Usage with values from Encryption
received_C = binascii.unhexlify("7297f66a875a6c3882776c52") # Example hex from enc
received_T = binascii.unhexlify("8519416174a811797e841280387431e6") 

try:
    plaintext = decrypt_modified_eax(key, nonce, ciphertext, hdr, tag)
    print(f"DECRYPTION SUCCESS:\nOriginal Message: {plaintext.decode()}")
except ValueError as e:
    print(e)